<a href="https://colab.research.google.com/github/sukhyun1017/pytorch-study/blob/main/pytorch_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1,1,28,28) # 배치크기,채널,높이,너비
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [3]:
conv1=nn.Conv2d(1,32,3,padding =1) #입력 채널, 출력채널, 커널사이즈
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [7]:
conv2=nn.Conv2d(32,64,kernel_size=(3,3),stride = (1,1),padding=(1,1))
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [9]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [12]:
out = pool(conv2(pool(conv1(inputs))))
print(out.shape)

torch.Size([1, 64, 7, 7])


In [13]:
out = out.view(out.size(0),-1) #첫번째 차원(배치크기)만 뺴고 펼치기
print(out.shape)

torch.Size([1, 3136])


cnn 으로 mnist

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 64

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [42]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [67]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
       
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, padding=1))

        self.layer4 = nn.Sequential(
            nn.Linear(4 * 4 * 128, 256, bias=True), #fully connected
            nn.ReLU(),
            nn.Dropout(p=0.2))
       
        self.fc1 = torch.nn.Linear(256, 10, bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.layer4(out)
        out = out.view(out.size(0), -1) 
        out = self.fc1(out)
        return out

In [68]:
model = CNN().to(device)

In [69]:
loss_fn = nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [56]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 937


In [70]:
for epoch in range(training_epochs):
  avg_cost =0
  for X,Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    H = model(X)
    cost = loss_fn(H,Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch #배치 하나의 평균 cost
  
  print('[Epoch: {:>4}] cost = {:>9}'.format(epoch+1,avg_cost))

[Epoch:    1] cost = 0.1585807502269745
[Epoch:    2] cost = 0.046376749873161316
[Epoch:    3] cost = 0.03317425027489662
[Epoch:    4] cost = 0.023993024602532387
[Epoch:    5] cost = 0.020576173439621925
[Epoch:    6] cost = 0.016117257997393608
[Epoch:    7] cost = 0.014118894934654236
[Epoch:    8] cost = 0.011649475432932377
[Epoch:    9] cost = 0.012217625975608826
[Epoch:   10] cost = 0.009947597980499268
[Epoch:   11] cost = 0.007572231814265251
[Epoch:   12] cost = 0.008433024398982525
[Epoch:   13] cost = 0.007917254231870174
[Epoch:   14] cost = 0.006019699852913618
[Epoch:   15] cost = 0.006014004349708557


In [75]:
with torch.no_grad():
  X_test = mnist_test.data.view(len(mnist_test),1,28,28).float().to(device) # view 함수
  Y_test = mnist_test.targets.to(device)
  
  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction,1) == Y_test #텐서 형태로 True,False 반환
  accuracy = correct_prediction.float().mean() #[1,1,1,0,0 .....1,1]들의 평균
  print('Accuracy',accuracy.item())

Accuracy 0.9628999829292297
